In [1]:
import mediapipe as mp
import cv2
import numpy as np
import pandas as pd
import pickle
import math
# Drawing helpers
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
# Determine important landmarks for squat
IMPORTANT_LMS = [
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_WRIST",
    "RIGHT_WRIST",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE"
]

# Generate all columns of the data frame

headers = ["label"] # Label column

for lm in IMPORTANT_LMS:
    headers += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]
    
def extract_important_keypoints(results) -> list:
    '''
    Extract important keypoints from mediapipe pose detection
    '''
    landmarks = results.pose_landmarks.landmark

    data = []
    for lm in IMPORTANT_LMS:
        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
        data.append([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])
    
    return np.array(data).flatten().tolist()


def rescale_frame(frame, percent=50):
    '''
    Rescale a frame to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

In [24]:
xaxis = np.array([[1, 0, 0]])
yaxis = np.array([[0, 1, 0]])
zaxis = np.array([[0, 0, 1]])
uaxis = np.array([[1, 1, 1]])
analyzed_results = {
     "torso": -1,
     "stance":-1,
     "grip":-1,
     "depth":-1,
     "left_hand_straight":-1,
     "right_hand_straight":-1,
     "state": -1  
     }    
def point_to_array(point) :
     return np.array([point.x, point.y, point.z, point.visibility])
def calculate_degree(pointX, pointY) : 
     return np.degrees(np.arccos(pointY @ pointX.T))
def check_perpendicular_limb(pointX, pointY, allowed_error = 15) :
     limb_xaxis_angle = np.degrees(np.arccos(pointX @ pointY.T))
     print(limb_xaxis_angle)
     if abs(limb_xaxis_angle - 90) > allowed_error:
        return False
     else:
        return True
def calculate_distance(pointX, pointY) :
     return np.linalg.norm(pointY - pointX)
def analyze_pose(results, visibility_threshold: int) : 
     landmarks = results.pose_landmarks.landmark
     
     left_shoulder = point_to_array(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value])
     right_shoulder = point_to_array(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value])
     left_hip = point_to_array(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value])
     right_hip = point_to_array( landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value])
     left_ankle = point_to_array(landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value])
     right_ankle = point_to_array( landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value])
     left_wrist = point_to_array(landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value])
     right_wrist = point_to_array( landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value])
     left_elbow = point_to_array(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value])
     right_elbow = point_to_array( landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value])
     left_knee = point_to_array(landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value])
     right_knee = point_to_array( landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value])
     if (left_shoulder[-1] < visibility_threshold or 
         right_shoulder[-1] < visibility_threshold or 
         left_hip[-1] < visibility_threshold or 
         right_hip[-1] < visibility_threshold or 
         left_ankle[-1] < visibility_threshold or
         right_ankle[-1] < visibility_threshold or
         left_wrist[-1] < visibility_threshold or
         right_wrist[-1] < visibility_threshold or
         left_elbow[-1] < visibility_threshold or
         right_elbow[-1] < visibility_threshold or
         left_knee[-1] < visibility_threshold or
         right_knee[-1] < visibility_threshold) :
        return analyzed_results
     #CHECK BODY STRAIGHT
     torso = np.array([(left_shoulder[:3] - left_hip[:3]) + (right_shoulder[:3] - right_hip[:3])])
     # torso = np.array(left_shoulder[:3] - left_hip[:3])
     # torso = np.array(right_shoulder[:3] - right_hip[:3])
     analyzed_results["torso"] = 0
     if not check_perpendicular_limb(pointX = torso, pointY = xaxis, allowed_error=5.):
          analyzed_results["torso"] = 1
     #CHECK STANCE
     hip_length = np.linalg.norm(left_hip[:3] - right_hip[:3])
     ankle_length = np.linalg.norm(left_ankle[:3] - right_ankle[:3])
     analyzed_results["stance"] = 0
     if ankle_length > hip_length*2.35 : 
          analyzed_results["stance"] = 1
     elif ankle_length < hip_length:
          analyzed_results["stance"] = 2
     #CHECK GRIP
     shoulder_width = np.linalg.norm(left_shoulder[:3] - right_shoulder[:3])
     grip_width = np.linalg.norm(left_wrist[:3] - right_wrist[:3])
     analyzed_results["grip"] = 0
     if grip_width > shoulder_width*1.85:
          analyzed_results["grip"] = 1
     elif grip_width < shoulder_width*1.1:
          analyzed_results["grip"] = 2
     #CHECK DEPTH
     hips = np.array([left_hip, right_hip])
     knees = np.array([left_knee, right_knee])
     h = ( calculate_distance(left_ankle[:3], left_knee[:3]) +  calculate_distance(right_ankle[:3], right_knee[:3]) ) /2
     margin = h * 0.25
     analyzed_results["depth"] = 0
     if(hips[:, 1] >= knees[:, 1] - margin).any() :
          analyzed_results["depth"] = 1
     #CHECK HAND STRAIGHT
     left_shoulder_to_wrist = left_wrist[:3] - left_shoulder[:3]
     right_shoulder_to_wrist = right_wrist[:3] - right_shoulder[:3]
     left_shoulder_to_elbow = left_elbow[:3] - left_shoulder[:3]
     right_shoulder_to_elbow = right_elbow[:3] - right_shoulder[:3]
     analyzed_results["left_hand_straight"] = 0
     analyzed_results["right_hand_straight"] = 0
     if(abs(calculate_degree(left_shoulder_to_wrist, left_shoulder_to_elbow)) > 15):
          analyzed_results["left_hand_straight"] = 1
     if(abs(calculate_degree(right_shoulder_to_wrist, right_shoulder_to_elbow)) > 15):
          analyzed_results["right_hand_straight"] = 1
     
     return analyzed_results
     

In [4]:
# with open("D:\\University\\Semester5\\PBL4-AIPT\\deadlift_model\\model\\deadlift.pkl", "rb") as f:
#     count_model = pickle.load(f)

In [25]:
video_path = 'D:\\University\\Semester5\\PBL4-AIPT\\Data\\Dataset\\deadlift\\deadlift_1.mp4'
cap = cv2.VideoCapture(video_path)

# Counter vars
VISIBILITY_THRESHOLD = 0.6

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, image = cap.read()

        if not ret:
            break
        
        # Reduce size of a frame
        image = rescale_frame(image, 50)

        # Recolor image from BGR to RGB for mediapipe
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)
        if not results.pose_landmarks:
            continue

        # Recolor image from BGR to RGB for mediapipe
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw landmarks and connections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=2), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=1))

        # Make detection
        try:
            # * Model prediction for SQUAT counter
            # Extract keypoints from frame for the input

            # Evaluate model prediction

            # Analyze squat pose
            analyzed_results= analyze_pose(results=results, visibility_threshold=VISIBILITY_THRESHOLD)

            torso_evaluation = analyzed_results["torso"]
            legs_evaluation = analyzed_results["stance"]
            grip_evaluation = analyzed_results["grip"]
            depth_evaluation = analyzed_results["depth"]
            left_hand_straight = analyzed_results["left_hand_straight"]
            right_hand_straight = analyzed_results["right_hand_straight"]
            state_evaluation = analyzed_results["state"]
            # * Evaluate FOOT PLACEMENT error
            if torso_evaluation == -1:
                torso_placement = "UNK"
            elif torso_evaluation == 0:
                torso_placement = "Correct"
            elif torso_evaluation == 1:
                torso_placement = "Curved"
            
            # * Evaluate KNEE PLACEMENT error
            if legs_evaluation == -1:
                legs_placement = "UNK"
            elif legs_evaluation == 0:
                legs_placement = "Correct"
            elif legs_evaluation == 1:
                legs_placement = "Too wide"
            elif legs_evaluation == 2:
                legs_placement = "Too tight"
            
            if grip_evaluation == -1:
                grip_placement = "UNK"
            elif grip_evaluation == 0:
                grip_placement = "Correct"
            elif grip_evaluation == 1:
                grip_placement = "Too wide"
            elif grip_evaluation == 2:
                grip_placement = "Too tight"
            
            # if left_hand_straight == -1 : 
            #     left_hand_placement = "UNK"
            # elif left_hand_straight == 0 : 
            #     left_hand_placement = "Correct"
            # elif left_hand_straight == 1 : 
            #     left_hand_placement = "Curved"
                
            # if right_hand_straight == -1 : 
            #     right_hand_placement = "UNK"
            # elif right_hand_straight == 0 : 
            #     right_hand_placement = "Correct"
            # elif right_hand_straight == 1 : 
            #     right_hand_placement = "Curved"
            # Visualization
            # Status box
            cv2.rectangle(image, (0, 0), (900, 60), (245, 117, 16), -1)

            # Display class
            cv2.putText(image, "TORSO", (10, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, torso_placement, (5, 40), cv2.FONT_HERSHEY_COMPLEX, .7, (255, 255, 255), 2, cv2.LINE_AA)

            # Display Foot and Shoulder width ratio
            cv2.putText(image, "LEGS", (200, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, legs_placement, (195, 40), cv2.FONT_HERSHEY_COMPLEX, .7, (255, 255, 255), 2, cv2.LINE_AA)

            # Display knee and Shoulder width ratio
            cv2.putText(image, "GRIP", (330, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, grip_placement, (325, 40), cv2.FONT_HERSHEY_COMPLEX, .7, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, "LEFT_HAND", (460, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(left_hand_straight), (455, 40), cv2.FONT_HERSHEY_COMPLEX, .7, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(image, "RIGHT_HAND", (590, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image,str(right_hand_straight), (585, 40), cv2.FONT_HERSHEY_COMPLEX, .7, (255, 255, 255), 2, cv2.LINE_AA)

        except Exception as e:
            print(f"Error: {e}")
        
        cv2.imshow("CV2", image)
        
        # Press Q to close cv2 window
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # (Optional)Fix bugs cannot close windows in MacOS (https://stackoverflow.com/questions/6116564/destroywindow-does-not-close-window-on-mac-using-python-and-opencv)
    for i in range (1, 5):
        cv2.waitKey(1)
  

d:\anacoda\envs\myenv\lib\site-packages\google\protobuf\symbol_database.py:78: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


[[89.91229351]]
[[89.93756864]]
[[89.95785773]]
[[89.97561281]]
[[89.98329336]]
[[89.98499066]]
[[89.98236104]]
[[89.98049298]]
[[89.98118966]]
[[89.98589907]]
[[89.99063581]]
[[89.9925107]]
[[89.99710058]]
[[90.00082987]]
[[90.00190221]]
[[90.00007172]]
[[90.00167681]]
[[90.00919002]]
[[90.02116334]]
[[90.01888889]]
[[89.98525021]]
[[89.97682517]]
[[89.9391874]]
[[89.93945378]]
[[89.9479198]]
[[89.96784347]]
[[89.98361096]]
[[90.00483919]]
[[90.08579745]]
[[90.15042826]]
[[90.18337382]]
[[90.22374391]]
[[90.27138834]]
[[90.26181231]]
[[90.26599584]]
[[90.25609197]]
[[90.26142981]]
[[90.2595993]]
[[90.21956721]]
[[90.19252973]]
[[90.14638136]]
[[90.10312908]]
[[90.09706045]]
[[90.09788007]]
[[90.10009989]]
[[90.09697849]]
[[90.10880498]]
[[90.08743329]]
[[89.99978485]]
[[89.97602945]]
[[89.91635406]]
[[89.8414472]]
[[89.79657946]]
[[89.70199067]]
[[89.63205518]]
[[89.62118812]]
[[89.61390014]]
[[89.5989246]]
[[89.59009978]]
[[89.58404466]]
[[89.56847481]]
[[89.55828046]]
[[89.54763187]

In [17]:
import numpy as np
x = np.array([1, 222, 3, 4])
y = np.array([-3, 0, 122])
# print(((x*y).sum()))
# print(x@y)
print(x[:3])

[  1 222   3]
